In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import math
from itertools import combinations_with_replacement
from collections import Counter
from scipy import signal
import plotly.express as px
import plotly.io as pio
import numpy as np
from scipy.interpolate import interpn
pio.templates.default = "plotly_dark"
GREEN = "LimeGreen"
RED = "crimson"

plt.style.use('fivethirtyeight')
np.random.seed(777)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [44]:
def plot_returns(returns):
    normalized_returns = returns / returns.iloc[0]

    fig = go.Figure()

    traces = []
    for column in normalized_returns.columns:
        traces.append(
            go.Scatter(
                x=normalized_returns.index,
                y=normalized_returns[column],
                mode="lines",
                name=column,
            )
        )

    layout = go.Layout(
        title="Normalized Returns Over Time",
        xaxis_title="Date",
        yaxis_title="Normalized Returns",
        width=1000,
        height=500,
        hovermode="x",
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)
    fig.show()


def align_data_range(stockData):

    start = stockData.index.min()
    end = stockData.index.max()
    for stock in stockData.columns.levels[1]:
        stock_start = stockData["Close"][stock].dropna().index.min()
        if stock_start > start:
            start = stock_start
            print(f"{stock} data starts at {start:%Y-%m-%d}. Adjusting start date.")
        stock_end = stockData["Close"][stock].dropna().index.max()
        if stock_end < end:
            end = stock_end
            print(f"{stock} data ends at {end:%Y-%m-%d}. Adjusting end date.")
    return stockData.loc[(stockData.index >= start) & (stockData.index <= end)]


def download_data(stocks, plot=False, **kwargs):

    stockData = yf.download(stocks, **kwargs)
    stockData["Close"] = stockData["Adj Close"]
    stockData = stockData.drop(columns=["Adj Close"])
    if isinstance(stockData.columns, pd.MultiIndex):
        stockData = align_data_range(stockData)
        stockData = stockData.swaplevel(axis=1)
    else:
        stockData.columns = pd.MultiIndex.from_product([[stocks[0]], stockData.columns])

    if plot:
        plot_returns(stockData)

    return stockData


def calculate_vwr(prices, n_trading_days=251.5, max_volatility=0.19, tau=2):
    log_returns = np.log(prices / prices.shift(1))
    n_periods = len(log_returns) - 1
    mean_return = np.log(prices.iloc[-1] / prices.iloc[0]) / n_periods

    log_returns.index
    time_step = pd.Series(range(len(log_returns)), index=log_returns.index)
    zero_variability_log_returns = np.exp(mean_return * time_step) * prices.iloc[0]

    diff = (prices - zero_variability_log_returns) / zero_variability_log_returns
    diff_std = diff.std()

    mean_log_return_normalized = (np.exp(mean_return * n_trading_days) - 1) * 100
    variability_factor = 1 - pow(diff_std / max_volatility, tau)
    vwr = mean_log_return_normalized * variability_factor
    return vwr, mean_log_return_normalized, variability_factor


def calculate_sharpe_ratio(prices, risk_free_rate=0.04, n_trading_days=251.5):
    risk_free_rate_per_day = pow(1 + risk_free_rate, 1 / n_trading_days) - 1
    risk_adjusted_returns = prices.pct_change() - risk_free_rate_per_day
    mean_return = risk_adjusted_returns.mean()
    std_dev = risk_adjusted_returns.std()
    sharpe_ratio = mean_return / std_dev

    returns = prices.pct_change()
    mean_return = returns.mean()
    returns_std_dev = returns.std()
    return sharpe_ratio, mean_return, returns_std_dev


def get_portfolio_metrics(weights, stock_data, risk_free_rate, n_trading_days=251.5):
    portfolio_weighted_prices = pd.Series(0, index=stock_data.index)
    for idx, stock in enumerate(stock_data.columns.levels[0]):
        portfolio_weighted_prices += stock_data[stock, "Close"] * weights[idx]
    sharpe_ratio, portfolio_mean_return, portfolio_std_dev = calculate_sharpe_ratio(
        portfolio_weighted_prices, risk_free_rate
    )
    vwr, mean_log_return_normalized, variability_factor = calculate_vwr(
        portfolio_weighted_prices
    )

    length = len(portfolio_weighted_prices)

    return {
        "length": length,
        "returns": portfolio_mean_return,
        "std_dev": portfolio_std_dev,
        "annualized_returns": portfolio_mean_return * n_trading_days,
        "annualized_std_dev": portfolio_std_dev * np.sqrt(n_trading_days / length),
        "sharpe_ratio": sharpe_ratio,
        "mean_log_return_normalized": mean_log_return_normalized,
        "variability_factor": variability_factor,
        "vwr": vwr,
    }


def n_combinations(n, r):
    result = math.factorial(n + r - 1) // (math.factorial(r) * math.factorial(n - 1))
    return result


def get_n_weight_combinations(stocks, num_portfolios):
    if len(stocks) == 1:
        return [np.array([1.0])]
    n_parts = len(stocks)
    while n_combinations(len(stocks), n_parts) < num_portfolios:
        n_parts += 1
    n_parts += 1
    portfolio_weights = []
    for combination in combinations_with_replacement(stocks, n_parts):
        counts = Counter(combination)
        portfolio_weights.append(np.array([counts[s] / n_parts for s in stocks]))
    return portfolio_weights


def get_portfolio_performances(numPortfolios, stock_data, risk_free_rate=0):
    tickers = stock_data.columns.levels[0]
    portfolio_weights = get_n_weight_combinations(tickers, numPortfolios)
    actual_num_portfolios = len(portfolio_weights)
    data = []
    for i in tqdm(range(actual_num_portfolios)):
        portfolio_metrics = get_portfolio_metrics(
            portfolio_weights[i], stock_data, risk_free_rate
        )
        portfolio_metrics["allocations"] = {
            asset: weight for asset, weight in zip(tickers, portfolio_weights[i])
        }
        data.append(portfolio_metrics)
    return pd.DataFrame(data)


def find_max_sharpe_portfolio(performances, verbose=False):
    selected = performances["sharpe_ratio"] == performances["sharpe_ratio"].max()

    if verbose:
        print("-" * 80)
        print(
            "Annualised Return:",
            round(performances.loc[selected, "annualized_returns"].iloc[0], 2),
        )
        print(
            "Annualised Volatility:",
            round(performances.loc[selected, "annualized_std_dev"].iloc[0], 2),
        )
        print("\n")
        allocations = {
            k: round(100 * v, 2)
            for k, v in performances.loc[selected, "allocations"].iloc[0].items()
        }
        display(pd.DataFrame(allocations, index=["Allocation"]))
    performances.loc[selected, "max_sharpe"] = True
    return performances


def find_min_volatility_portfolio(performances, verbose=False):
    selected = (
        performances["annualized_std_dev"] == performances["annualized_std_dev"].min()
    )

    if verbose:
        print("-" * 80)
        print("Minimum Volatility Portfolio Allocation\n")
        print(
            "Annualised Return:",
            round(performances.loc[selected, "annualized_returns"].iloc[0], 2),
        )
        print(
            "Annualised Volatility:",
            round(performances.loc[selected, "annualized_std_dev"].iloc[0], 2),
        )
        print("\n")
        allocations = {
            k: round(100 * v, 2)
            for k, v in performances.loc[selected, "allocations"].iloc[0].items()
        }
        display(pd.DataFrame(allocations, index=["Allocation"]))
    performances.loc[selected, "min_volatility"] = True
    return performances


def find_efficient_portfolios(performances, n_bins):
    min_std_dev = performances.loc[
        performances["min_volatility"] == True, "annualized_std_dev"
    ].min()
    next_smallest = performances.loc[
        performances["annualized_std_dev"] > min_std_dev, "annualized_std_dev"
    ].min()

    max_std_dev = performances.loc[
        performances["max_sharpe"] == True, "annualized_std_dev"
    ].max()
    next_largest = performances.loc[
        performances["annualized_std_dev"] < max_std_dev, "annualized_std_dev"
    ].max()

    if (
        not np.isnan(next_smallest)
        and not np.isnan(next_largest)
        and not next_smallest >= next_largest
    ):
        bins = np.linspace(next_smallest, next_largest, n_bins + 1)
        performances["vol_bin"] = pd.cut(performances["annualized_std_dev"], bins)

        performances["efficient"] = performances.groupby("vol_bin", observed=True)[
            "annualized_returns"
        ].transform(lambda x: x == x.max())

    performances.loc[
        (performances["min_volatility"] == True) | (performances["max_sharpe"] == True),
        "efficient",
    ] = True

    return performances


def auto_round_off(series):
    string = str(series.diff().mean())
    if "e-" in string:
        decimal_places = int(string.split("e-")[1])
    elif "." in string:
        string = string.split(".")[1]
        decimal_places = len(string) - len(string.lstrip("0")) + 1
    return series.round(decimal_places).astype(str)


def plot_allocations(results):
    efficient_portfolios = results.loc[results["efficient"] == True].copy()
    if "vol_bin" in efficient_portfolios.columns:
        efficient_portfolios["annualized_std_dev"] = (
            efficient_portfolios["vol_bin"]
            .apply(lambda x: x.mid)
            .astype(float)
            .fillna(efficient_portfolios["annualized_std_dev"])
        )
    efficient_portfolios["volatility_str"] = auto_round_off(
        efficient_portfolios["annualized_std_dev"]
    )

    efficient_portfolios["allocations"] = efficient_portfolios["allocations"].apply(
        lambda x: list(x.items())
    )
    efficient_portfolios = efficient_portfolios.explode("allocations")
    efficient_portfolios["asset"] = efficient_portfolios["allocations"].str[0]
    efficient_portfolios["weight"] = efficient_portfolios["allocations"].str[1]

    allocations = px.bar(
        efficient_portfolios,
        x="volatility_str",
        y="weight",
        labels="annualized_std_dev",
        color="asset",
    )

    min_volatility = go.Scatter(
        x=efficient_portfolios.loc[
            efficient_portfolios["min_volatility"] == True, "volatility_str"
        ].iloc[:1],
        y=[1.1],
        mode="markers",
        marker=dict(symbol="star", color=GREEN, size=10),
        name="Min Volatility",
        hoverinfo="none",
    )
    max_sharpe = go.Scatter(
        x=efficient_portfolios.loc[
            efficient_portfolios["max_sharpe"] == True, "volatility_str"
        ].iloc[-1:],
        y=[1.1],
        mode="markers",
        marker=dict(symbol="star", color=RED, size=10),
        name="Max Sharpe Ratio",
        hoverinfo="none",
    )

    layout = go.Layout(
        title="Efficient Portfolio Allocations",
        xaxis=dict(
            title="annualized_std_dev",
            categoryorder="array",
            categoryarray=efficient_portfolios["volatility_str"],
        ),
        yaxis=dict(title="Allocations"),
        width=1000,
        height=400,
        barmode="stack",
        hovermode="x",
    )
    fig = go.Figure(
        data=[*allocations.data],
        layout=layout,
    )
    fig.update_traces(hovertemplate="%{y:.2%}")
    fig.add_trace(min_volatility)
    fig.add_trace(max_sharpe)

    fig.show()


def get_label(row):
    _return = row["annualized_returns"]
    volatility = row["annualized_std_dev"]
    vwr = row["vwr"]
    sharpe_ratio = row["sharpe_ratio"]
    allocations = [
        f"&nbsp;<i>{k}</i> - {v*100:.0f}%"
        for k, v in row["allocations"].items()
        if v != 0
    ]
    allocations = "<br>".join(allocations)
    return (
        f"<b>Volatility : </b>{volatility*100:.2f}<br>"
        f"<b>Return : </b>{_return:.2%}<br>"
        f"<b>VWR : </b>{vwr:.2f}<br>"
        f"<b>Sharpe ratio : </b>{sharpe_ratio:.2%}%<br>"
        "<b>Allocations : </b><br>"
        f"{allocations}"
    )


def get_kde(x, y, bins=20):
    data, x_e, y_e = np.histogram2d(x, y, bins=bins, density=True)
    z = interpn(
        (0.5 * (x_e[1:] + x_e[:-1]), 0.5 * (y_e[1:] + y_e[:-1])),
        data,
        np.vstack([x, y]).T,
        method="splinef2d",
        bounds_error=False,
    )
    z[np.where(np.isnan(z))] = np.min(z)
    return z


def plot_efficient_frontier(
    results, risk_free_rate, reward_col, risk_col, combined_metric_col
):
    if len(results) > 5_000:
        density = get_kde(results[reward_col], results[risk_col])
        weights = 1 / density
        sampled = results.sample(weights=weights, n=5_000)
        sampled_results = pd.concat(
            [results.loc[results["efficient"] == True], sampled]
        )
    else:
        sampled_results = results

    labels = sampled_results.apply(get_label, axis=1)
    simulated_portfolios = go.Scatter(
        x=sampled_results[risk_col],
        y=sampled_results[reward_col],
        mode="markers",
        text=labels,
        hoverinfo="text",
        marker=dict(
            color=sampled_results[combined_metric_col],
            colorscale="plasma",
            size=10,
            opacity=0.8,
            colorbar=dict(title="Variability Adjusted Returns"),
        ),
        name="Simulated Portfolios",
    )
    efficient_rows = results.loc[results["efficient"] == True]
    efficient_frontier = go.Scatter(
        x=efficient_rows[risk_col],
        y=signal.savgol_filter(
            efficient_rows[reward_col],
            len(efficient_rows),
            min(5, len(efficient_rows) - 1),
        ),
        line=dict(color="white", width=5),
        mode="lines",
        name="Efficient Frontier",
    )

    max_sharpe = go.Scatter(
        x=results.loc[results["max_sharpe"] == True, risk_col],
        y=results.loc[results["max_sharpe"] == True, reward_col],
        mode="markers",
        marker=dict(symbol="star", color=RED, size=20),
        name="Max Sharpe Ratio",
        hoverinfo="skip",
    )

    min_volatility = go.Scatter(
        x=results.loc[results["min_volatility"] == True, risk_col],
        y=results.loc[results["min_volatility"] == True, reward_col],
        mode="markers",
        marker=dict(symbol="star", color=GREEN, size=20),
        name="Min Volatility",
        hoverinfo="skip",
    )
    mostly_smaller_than_two = (
        len(results.loc[results[reward_col] < 2]) / len(results) > 0.5
    )
    layout = go.Layout(
        title="Simulated Portfolio Performance",
        xaxis=dict(title=risk_col, tickformat=".2%"),
        yaxis=dict(
            title=reward_col, tickformat=".2%" if mostly_smaller_than_two else None
        ),
        width=1000,
        height=700,
    )

    fig = go.Figure(
        data=[simulated_portfolios, efficient_frontier, max_sharpe, min_volatility],
        layout=layout,
    )
    if combined_metric_col == "sharpe":
        fig.add_hline(
            y=risk_free_rate,
            line_width=3,
            line_dash="dash",
            line_color="LightGrey",
            annotation_text="Risk Free Rate",
        )
    fig.update_layout(legend_orientation="h")

    fig.show()
    return None


def simulate_efficient_frontier(
    stock_data, num_portfolios, risk_free_rate, plot, n_trading_days=251.5
):
    performances = get_portfolio_performances(
        num_portfolios, stock_data, risk_free_rate
    )
    performances = performances.sort_values(by="annualized_std_dev")

    performances = find_max_sharpe_portfolio(performances, verbose=plot)
    performances = find_min_volatility_portfolio(performances, verbose=plot)
    performances = find_efficient_portfolios(
        performances,
        n_bins=30,
    )
    if plot:
        plot_efficient_frontier(
            performances,
            risk_free_rate,
            reward_col="mean_log_return_normalized",
            risk_col="variability_factor",
            combined_metric_col="vwr",
        )
        plot_allocations(performances)
    return performances


num_portfolios = 25_00
risk_free_rate = 0.04

stocks = [
    "VTI",  # Total stock Market
    "BND",  # Total Bond Market
    "VEA",  # International Developed Markets
    "VB",  # Small Cap
    "VTV",  # Value
]


stock_data = download_data(stocks, plot=False, start="2012-01-01", end="2024-10-31")
results = simulate_efficient_frontier(
    stock_data, num_portfolios, risk_free_rate, plot=True
)

[*********************100%***********************]  5 of 5 completed


  0%|          | 0/3876 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
Annualised Return: 0.15
Annualised Volatility: 0.0




,BND,VB,VEA,VTI,VTV
Allocation,0.0,0.0,0.0,100.0,0.0


--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 0.03
Annualised Volatility: 0.0




,BND,VB,VEA,VTI,VTV
Allocation,93.33,0.0,0.0,0.0,6.67


## Multi period

In [ ]:
def plot_multi_period_results(results_df):
    for column in ["std_dev", "returns", "sharpe_ratio"]:
        fig = px.line(
            results_df,
            x=results_df.index,
            y=column,
            title=f"Optimal portfolio {column}",
            width=1000,
            height=300,
        )
        fig.show()

    allocation_df = results_df[["allocations"]]
    allocation_df = allocation_df["allocations"].apply(pd.Series)
    fig = px.line(
        allocation_df,
        x=allocation_df.index,
        y=allocation_df.columns.tolist(),
        title="Optimal Portfolio Allocations",
        width=1000,
        height=500,
    )
    fig.show()

num_portfolios = 25_00
risk_free_rate = 0.04

stocks = [
    "VTI",  # Total stock Market
    "BND",  # Total Bond Market
    "VEA",  # International Developed Markets
    "VB",  # Small Cap
    "VTV",  # Value
]

all_results = []
for today in [f"{year}-{month:02}-01" for year in range(2012, 2025) for month in [1]]:
    start = "2007-07-26"
    end = today
    if end <= start:
        continue

    mean_returns, cov_matrix = get_returns_and_covariance(
        stocks, plot=False, start=start, end=end
    )
    results = simulate_efficient_frontier(
        mean_returns, cov_matrix, num_portfolios, risk_free_rate, plot=True
    )
    max_sharpe = results.loc[results["max_sharpe"] == True].iloc[0]

    mean_returns, cov_matrix = get_returns_and_covariance(
        stocks, plot=False, start=today, end="2024-10-30"
    )
    portfolio_std_dev, portfolio_return = portfolio_annualised_performance(
        np.array(list(max_sharpe["allocations"].values())), mean_returns, cov_matrix
    )

    all_results.append(
        dict(
            today=today,
            std_dev=portfolio_std_dev,
            returns=portfolio_return,
            allocations=max_sharpe["allocations"],
            sharpe_ratio=(portfolio_return - risk_free_rate) / portfolio_std_dev,
        )
    )
results_df = pd.DataFrame(all_results)
results_df.set_index("today", inplace=True)

plot_multi_period_results(results_df)

In [ ]:


efficient_portfolios = results.loc[results["efficient"] == True].copy()
if "vol_bin" in efficient_portfolios.columns:
    efficient_portfolios["annualized_std_dev"] = (
        efficient_portfolios["vol_bin"]
        .apply(lambda x: x.mid)
        .astype(float)
        .fillna(efficient_portfolios["annualized_std_dev"])
    )

auto_round_off(
    efficient_portfolios["annualized_std_dev"]
    )

2      0.00086
9       0.0009
3      0.00097
14     0.00103
10     0.00111
13     0.00117
26     0.00124
12     0.00132
33     0.00138
28     0.00146
32     0.00152
69      0.0016
68     0.00166
62     0.00174
61     0.00181
66     0.00188
65     0.00194
123    0.00202
122    0.00208
121    0.00215
120    0.00222
199     0.0023
205    0.00236
204    0.00243
203    0.00251
326    0.00258
324    0.00264
322    0.00272
491    0.00278
489    0.00286
487    0.00292
486    0.00296
Name: annualized_std_dev, dtype: object